In [ ]:
import numpy as np
import pandas as pd
#넘파이와 판다스의 분산은 다르게 계산됨
#판다스는 자유도가-1되어있음

In [ ]:
print('넘파이')
print(np.arange(1,11).mean())
print(np.arange(1,11).var())
print(np.sum((np.arange(1,11) - np.arange(1,11).mean())**2)/9)

print()
print('판다스')
print(pd.DataFrame(data=range(1,11)).mean())
print(pd.DataFrame(data=range(1,11)).var())

In [ ]:
import requests
import pymysql

url = f"http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"
data = requests.post(url).text

In [ ]:
sql = """
create TABLE IF NOT EXISTS st_master2(
    ISU_CD varchar(12) not null,
    ISU_SRT_CD varchar(10) not null,
    ISU_NM varchar(100) not null,
    ISU_ABBRV varchar(100) not null,
    ISU_ENG_NM varchar(100) not null,
    LIST_DD date not null,
    MKT_TP_NM varchar(15) not null,
    SECUGRP_NM varchar(20) not null,
    SECT_TP_NM varchar(30) not null,
    KIND_STKCERT_TP_NM varchar(15) not NULL,
    PARVAL int null default null,
    IST_SHRS bigint not null,
    PRIMARY KEY(ISU_SRT_CD)
)
"""


In [ ]:
import pymysql
conn = pymysql.connect(host='127.0.0.1', user='play', passwd='123', database='sk17', port=3306)
cur = conn.cursor()
cur.execute(sql)

In [ ]:
cur.execute('show tables')
cur.fetchall()
cur.execute('desc st_master2')
cur.fetchall()

In [ ]:
modify_sql = """ALTER TABLE st_master2
MODIFY COLUMN PARVAL float default null"""
cur.execute(modify_sql)
cur.execute('desc st_master2')
cur.fetchall()

In [ ]:
import requests
import pymysql

headers1 = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Length': '88',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Cookie': '__smVisitorID=jMYvcn8H966; _ga=GA1.1.7639877.1748567776; _ga_Z6N0DBVT2W=GS2.1.s1748567775$o1$g1$t1748567823$j12$l0$h0; JSESSIONID=g8b2nahgZKunheR73JiFUWtqaaPK9Fmw5b41wPRhG396lt7SlJErT5itRiUjyDuC.bWRjX2RvbWFpbi9tZGNvd2FwMi1tZGNhcHAxMQ==; _ga_808R2EHLL3=GS2.1.s1748567823$o1$g0$t1748567836$j47$l0$h0',
    'Host': 'data.krx.co.kr',
    'Origin': 'http://data.krx.co.kr',
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020101',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}
payload = {
    'bld': 'dbms/MDC/STAT/standard/MDCSTAT01901',
    'locale': 'ko_KR',
    'mktId': 'ALL',
    'share': '1',
    'csvxls_isNo': 'false'
}

url = f"http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"
r = requests.post(url, data = payload, headers = headers1)

In [ ]:
sql = "insert IGNORE INTO st_master values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
for x in r.json()['OutBlock_1']:
    ISU_CD, ISU_SRT_CD, ISU_NM, ISU_ABBRV, ISU_ENG_NM, LIST_DD, MKT_TP_NM, SECUGRP_NM, SECT_TP_NM, KIND_STKCERT_TP_NM, PARVAL, LIST_SHRS = x.values()
    LIST_SHRS = LIST_SHRS.replace(",", "")
    PARVAL = None if PARVAL == "무액면" else PARVAL.replace(",", "")
    cur.execute(sql,[ISU_CD, ISU_SRT_CD, ISU_NM, ISU_ABBRV, ISU_ENG_NM, LIST_DD, MKT_TP_NM, SECUGRP_NM, SECT_TP_NM, KIND_STKCERT_TP_NM, PARVAL, LIST_SHRS] )x`


conn.commit()
#conn.close()

In [ ]:
def get_sql(sql):
    cur.execute(sql)
    return cur.fetchall()

In [ ]:
rt = get_sql(""" select isu_srt_cd from st_master where MKT_TP_NM in ('KOSPI') order by list_dd desc """)

In [ ]:
rt.__len__()

In [ ]:
import pymysql

conn = pymysql.connect(host='127.0.0.1', user='play', passwd='123', database='sk17', port=3306)
cur = conn.cursor()

In [ ]:
import pandas as pd
import streamlit

rt = get_sql(""" select * from st_master where MKT_TP_NM in ('KOSPI') order by list_dd desc """)
st.write(pd.DataFrame(rt, columns=[x[0] for x in get_sql("desc st_master")]))

In [ ]:
data